In [ ]:
import pandas as pd 
import numpy as np
import os
import keras_preprocessing
import random
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBClassifier
from google.colab import files 

In [ ]:
import process_tool
import DLmodels
import preprocess 
from process_tool import empty_col
from process_tool import name_to_num
from process_tool import num_to_name
from process_tool import array_valid

In [ ]:
wealth_total = pd.read_csv("./data/wealth_total.csv")

In [ ]:
safe_sa = pd.read_pickle("./data/safe_street_aerial.pkl")
wealth_sa = pd.read_pickle("./data/wealth_street_aerial.pkl")
lively_sa = pd.read_pickle("./data/lively_street_aerial.pkl")
safe_sa = preprocess.dense_preprocess(safe_sa, 'aerial', 'dense_aerial')
wealth_sa = preprocess.dense_preprocess(wealth_sa, 'aerial', 'dense_aerial')
lively_sa = preprocess.dense_preprocess(lively_sa, 'aerial', 'dense_aerial')

In [ ]:
X1 = safe_sa['dense_aerial']
X1 = array_valid(X1) #np.ndarray processing 
safe_aerial_feature = DLmodels.dense_tb(X1)

X2 = wealth_sa['dense_aerial']
X2 = array_valid(X2)
wealth_aerial_feature = DLmodels.dense_tb(X2)

X3 = lively_sa['dense_aerial']
X3 = array_valid(X3)
lively_aerial_feature = DLmodels.dense_tb(X3)

#save the feature extracted by denseNet from aerial images 
safe_aerial_feature.to_csv("./data_collection_aug/aerial_features/safe_aerial_feature_dense.csv")
wealth_aerial_feature.to_csv("./data_collection_aug/aerial_features/wealth_aerial_feature_dense.csv")
lively_aerial_feature.to_csv("./data_collection_aug/aerial_features/lively_aerial_feature_dense.csv")

# train the safety perception model on aerial features

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

X = safe_aerial_feature
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = safe_total['class']
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.25)

#from hyperparam tuning, we found that default param is quite close to the optimum so we just used the default
clf = SVC(probability = True)
clf.fit(x_train, y_train)
pred_clf = clf.predict(x_test)
pred_clf2 = clf.predict(x_train)
acc_clf = accuracy_score(pred_clf, y_test)
acc_clf2 = accuracy_score(pred_clf2, y_train)
print(accuracy_score(pred_clf2, y_train), acc_clf, f1_score(pred_clf2, y_train), f1_score(pred_clf, y_test))

#save the trained model 
import pickle
filename = './data/models/svm_safe_aerial_model.sav'
pickle.dump(clf, open(filename, 'wb'))

# train the lively perception model on aerial features

In [ ]:
X = lively_aerial_feature
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = lively_sa['class']
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.25)

clf = SVC(probability = True)
clf.fit(x_train, y_train)
pred_clf = clf.predict(x_test)
pred_clf2 = clf.predict(x_train)
acc_clf = accuracy_score(pred_clf, y_test)
acc_clf2 = accuracy_score(pred_clf2, y_train)
print(accuracy_score(pred_clf2, y_train), acc_clf, f1_score(pred_clf2, y_train), f1_score(pred_clf, y_test))

filename = './data/models/svm_lively_aerial_model.sav'
pickle.dump(clf, open(filename, 'wb'))

# train the wealth perception model on lively features

In [ ]:
X = wealth_aerial_feature
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = wealth_sa_nona['class']
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.25)

clf = SVC(probability = True)
clf.fit(x_train, y_train)
pred_clf = clf.predict(x_test)
pred_clf2 = clf.predict(x_train)
acc_clf = accuracy_score(pred_clf, y_test)
acc_clf2 = accuracy_score(pred_clf2, y_train)
print(accuracy_score(pred_clf2, y_train), acc_clf, f1_score(pred_clf2, y_train), f1_score(pred_clf, y_test))

filename = './data/models/svm_wealth_aerial_model.sav'
pickle.dump(clf, open(filename, 'wb'))